# Custom Layers

Creating a Keras model with multiple outputs and a custom sampling layer, similar to those used in Variational AutoEncoders (VAEs), is an interesting task. Additionally, we'll incorporate custom loss functions for each output. Here's how we can achieve this:

Here's how we can do it:

1. Define the Custom Sampling Layer: This layer will be used to sample from a probability distribution, typically used in VAEs.

1. Define the Model Architecture: The model will include the custom sampling layer and have two outputs.

1. Implement Custom Loss Functions:
        - Custom Binary Classification Loss: A simple example could be a variant of binary cross-entropy.
        - KL Divergence Loss: TensorFlow provides a function for KL Divergence, which we could use directly but in this case we will provide a variation

1. Generate Dummy Data: We'll create dummy data appropriate for our model's input and output specifications.

1. Compile and Train the Model: We'll compile the model with our custom loss functions and then train it.



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import KLDivergence
import numpy as np

# 1. Custom Sampling Layer

class SamplingLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = None  # Extract z_mean, z_log_var
        batch = None  # Calculate batch_size from the shape of z_mean
        dim = None   # Calculate dimension  from the shape of z_mean
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return None   # Return the sampled data z_mean + e^(0.5*z_log_var)*epsilon



In [ ]:
# 2. Model architecture
input_layer = Input(shape=(10,))
dense_layer = Dense(64, activation='relu')(input_layer)

# For the VAE-like output
z_mean = None  # Dense layer with 10 neurons
z_log_var = None   # Dense layer with 10 neurons
sampling_output = None  # Custom sampling layer
output1 = None  # Binary classification output from dense_layer
output2 = None  # Multiclass classification output using the sampled data

model = None  # Create the model

In [ ]:
# 3. Custom binary classification loss
def custom_binary_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    epsilon = 1e-15
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
    return -tf.reduce_mean(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))

# 3. Custom KL Divergence Loss
def custom_kl_divergence_loss(y_true, y_pred, scale_factor=1.0):
    kl_loss = tf.keras.losses.KLDivergence()(y_true, y_pred)
    return scale_factor * kl_loss


In [ ]:
# 4. Generate dummy data
x_dummy = np.random.random((1000, 10))
y_dummy_output1 = np.random.randint(2, size=(1000, 1))
y_dummy_output2 = np.random.randint(5, size=(1000, 5))


In [ ]:
# 5. Compile the model
model.compile(optimizer='adam',
              loss={'output1': custom_binary_loss, 'output2': lambda y_true, y_pred: custom_kl_divergence_loss(y_true, y_pred, scale_factor=2.0)},
              metrics={'output1': ['accuracy'], 'output2': ['accuracy']})

# 5. Train the model
model.fit(x_dummy, {'output1': y_dummy_output1, 'output2': y_dummy_output2}, epochs=10)

Once a Keras model is trained, you can evaluate its performance on a test dataset and use it to make predictions. Continuing from the previous example, I'll show you how to:

1. Evaluate the Model: We'll evaluate the model on a separate set of dummy data to see how it performs.

2. Use the Model for Prediction: We'll use the model to make predictions based on new input data.

In [ ]:
# 1. Evaluate the model

# Generate some dummy test data
x_dummy_test = np.random.random((200, 10))
y_dummy_test_output1 = np.random.randint(2, size=(200, 1))  # Binary labels
y_dummy_test_output2 = np.random.randint(5, size=(200, 5))  # One-hot encoded labels for 5 classes

# Evaluate the model
evaluation = model.evaluate(x_dummy_test, {'output1': y_dummy_test_output1, 'output2': y_dummy_test_output2})
print(f"Test Loss, Test Accuracy for Output 1: {evaluation[1]}, {evaluation[3]}")
print(f"Test Loss, Test Accuracy for Output 2: {evaluation[2]}, {evaluation[4]}")


In [ ]:
# 2. Use the model for prediction

# New sample data for prediction
new_sample = np.random.random((1, 10))

# Making predictions
predictions = model.predict(new_sample)
print(f"Predictions for Output 1 (Binary classification): {predictions[0]}")
print(f"Predictions for Output 2 (Multiclass classification): {predictions[1]}")
